<a href="https://colab.research.google.com/github/anandraiyer/access_forums_eval/blob/main/annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/final.csv')

In [ ]:
data.columns

In [ ]:
#Clean up post text attribute
#TODO : Check if this causes any inline parent post / quoted text to be lost
data['Post'] = data['Post'].str.split('-------------- next part --------------').str[0]
data['Post'] = data['Post'].str.split('An HTML attachment was scrubbed').str[0]
data['Post'] = data['Post'].str.split('--------------------------------------------------').str[0]

In [ ]:
%pip install fuzzywuzzy
%pip install python-Levenshtein

In [ ]:
from fuzzywuzzy import fuzz
#helps with partial text matching

In [ ]:
dic_parentpost = {}
min_threads = min(list(data['ThreadID']))
max_threads = max(list(data['ThreadID']))

#Iterate through every Thread
for i in range(min_threads,max_threads+1):
  if i%1000 == 0:
      print(i)
      #Prints an update once every 1000 threads
  
  #Every Thread is loaded into temp dataframe
  temp = data[data['ThreadID']==i]

  thread_name = list(temp['Thread'])[0] #Thread Name Text
  #Attributes of a post in a Thread
  datetime = list(temp['DateTime']) # Datetime of Post
  authors = list(temp['Author']) # Author name of a Post
  posts = list(temp['Post']) # Body of the Post / Message Text of a Post
  parents = list(temp['ParentPosts']) #Quoted Text in a Post
  postid = list(temp['PostID']) # Global PostID

  #Dictionary with PostID as key and Post Attribute as Value
  dic_datetime = {}
  dic_authors = {}
  dic_posts = {}
  dic_parents = {}
  dic_postid = {}

  #Load Dictionary for the Thread so we can identify attributes of a post with relative postID (eg: Post 0 of a Thread has the following Date etc.)
  for j in range(len(postid)):
    dic_datetime[j] = datetime[j]
    dic_authors[j] = authors[j]
    dic_posts[j] = posts[j]
    dic_parents[j] = parents[j]
    dic_postid[j] = postid[j]

  #j identifies the current post id we are trying to find the post ID of the Parent for, from the data in ParentPost Text or data in Post Text if no ParentPost exists
  for j in range(1,len(postid)):
    if (type(dic_parents.get(j)) == float):
      #If no parent post / quoted text field for post j exists ie. it is a nan (data type is float)
      PARENTS_NULL = True
      if (type(dic_posts.get(j)) == float):
        #If the post j is nan / null 
        POSTS_NULL = True
        #print('Null Values for both parents and post : ',i,j)
        po = ''
        pp = ''
      else:
        #Parent Post text field for post j is Null / Nan but Post j is not null
        po = dic_posts.get(j)
        pp = ''
    else:
      #Parent Post Text field for post j is not Null / Nan
      if (type(dic_posts.get(j)) == float):
        #Parent Post Text Field is not Null but the Post j is Null
        POSTS_NULL = True
        po = ''
        pp = dic_parents.get(j).replace('-----','').replace('Original Message','')
      else:
        #Post j and Parent Post text field are not null
        po = dic_posts.get(j)
        pp = dic_parents.get(j).replace('-----','').replace('Original Message','')
    #For every post j, we iterate over all previous posts in the thread in reverse order from j-1 to 0 
    #to check if post k is parent of j based on matching post attributes of k and j
    for k in range(j-1,-1,-1):
      #We use the dictionary to get Post Attributes for post k
      cpost = dic_posts.get(k)
      cauthor = dic_authors.get(k)
      cdate = dic_datetime.get(k)
      #2021-12-13 11:01:33.123456 -> format for date time attribute (it is stored as a string)
      chours = cdate[10:14].strip()
      cminutes = cdate[13:16].strip()
      #print(cdate,chours,cminutes)

      if type(cpost) == float:
        #print('Post is Null.',i,j,k)
        continue
      if type(cauthor) == float:
        #print('Author is Null.',i,j,k)
        continue
      #Attributes we will match between post j and post k to check if post k is the parent to post j
      #Based on the text in the ParentPost field of post j and if no ParentPost field available the Post field of post j
      POST_MATCH = False 
      AUTHOR_MATCH = False
      AUTHOR_POST = False
      TIME_MATCH = False

      #Check if Parent Post field is atleast greater than 3 chars long 'Nan' 
      if len(pp) > 3:
        #Check if the message body of post k is in the ParentPost field of post j
        #Set Flag POST_MATCH true if this is the case
        if cpost.lower() in pp.lower():
          POST_MATCH = True
        #Check if Hour attribute or Minutes attribute from time stamp of post k are present in ParentPost field of post j
        #We use the first word in the author name to test in case of a coincidence where another post may have similar timestamp
        #We also check if there is atleast an 80% fuzzy match between message body of post k and ParentPost text in post j
        #Set Flag TIME_MATCH true if this is the case
        #TODO : Check if chours match is accurate for cases of TimeZone difference between post k and post j
        if ((chours in pp) or (cminutes in pp)) and (cauthor.split()[0].lower() in pp.lower()) and (fuzz.token_set_ratio(cpost.lower()[:len(cpost)//2],pp.lower().split('From:')[0])>80):
          TIME_MATCH = True

        #Get word tokens from ParentPost text field of post j
        words = pp.split()

        #For Author Name of post k
        for u in cauthor.lower():
          #Break author name into tokens and check if any token is in the ParentPost field. eg 'Brian Vogel' becomes 'brian' and 'vogel'
          #Set AUTHOR_MATCH as True
          if u in pp.lower():
            AUTHOR_MATCH = True
        #Parent Posts that Start with the token / word 'From:'    
        if words[0] == 'From:':
          #If the first token in Author Name of post k is present in the Parent Post text of post j before the word 'sent' and is atleast 2 chars long
          #Set AUTHOR_MATCH token as true
          if (cauthor.split()[0].lower() in pp.lower().split('sent')[0]) and len(cauthor.split()[0])>2:
            AUTHOR_MATCH = True
        #Parent Posts that Start with the token / word 'On'
        elif words[0] == 'On':
          #Check if first token in Author Name of post k is present in the Parent Post text of post j before the word 'wrote'
          #Set AUTHOR_MATCH as true 
          if (cauthor.split()[0].lower() in pp.lower().split('wrote')[0]) and len(cauthor.split()[0])>2:
            AUTHOR_MATCH = True
        else:
          #All other types of Parent Post text field for post j
          #Check if author name of post k is in parent post text field of post j
          if cauthor.lower() in pp.lower():
            AUTHOR_MATCH = True
      else:
        #Case that Prof IV was interested in during the meeting on Dec 12,2021 <---- ATTN!!!
        #If there is no Parent Post text for post j we check if Quoted Text is present in the Post body of post j
        #TODO : Test this case -> look for examples and how the code identifies the annotation
        if cauthor.lower() in po.lower():
          #If the author name of post k is in the post message text of post j then set AUTHOR_POST flag true
          AUTHOR_POST = True

      #If we have AUTHOR_MATCH flag true but the TIME_MATCH and AUTHOR_POST flag do not match (so that we do not match another post by same author as parent)
      #We use the costlier (in terms of time) / slower fuzzy matching step 
      if AUTHOR_MATCH == True and TIME_MATCH == False and AUTHOR_POST == False:
        # lev_ratio = fuzz.partial_ratio(cpost.lower(),pp.lower())
        lev_ratio_1 = fuzz.token_set_ratio(cpost.lower()[:len(cpost)//2],pp.lower().split('From:')[0])
        #It matches the tokens in the post body of post k as a set and the parent post text in post j prior to From: token as a set

        if lev_ratio_1 < 85:
          #If this ratio is less than 85% match we cannot say we have an author match
          #TODO find examples where this threshold fails
          AUTHOR_MATCH = False

      #Finally we can assign values
      #If atleast one of the 4 test flags are true we can assign post k as the parent of post j
      if POST_MATCH == True or AUTHOR_MATCH == True or TIME_MATCH == True or AUTHOR_POST == True:
        #print('Match : ',i,j,k)
        dic_parentpost[dic_postid.get(j)] = dic_postid.get(k)
        #For the key postid j we assign parent postid k and we break to go to next post j in search of a parent
        break

    #DEFAULT CASE : Assign the previous post j-1 as the parent post for post j if there is no match for any previous post 0,...,j-1 for post j
    #TODO : Discuss with Vikas if this case can be removed
    if POST_MATCH == False and AUTHOR_MATCH == False and TIME_MATCH == False and AUTHOR_POST == False:
      #print('Default : ',i,j,j-1)
      dic_parentpost[dic_postid.get(j)] = dic_postid.get(j-1)

In [ ]:
parent_id_list = []
a = list(data['PostID'])
for i in range(len(a)):
  parent_id_list.append(dic_parentpost.get(a[i],-1))

In [ ]:
data['ParentID_List'] = parent_id_list

In [ ]:
#Final Forum Dataset with annotations
data